In [1]:
from acados_template import AcadosModel, AcadosOcp, AcadosOcpSolver
from casadi import SX, vertcat, cross
import numpy as np

In [2]:
Tmax = 0.000953
Wmax = 1570.79632679
Vmax = 6
a_1 = np.array([[1],[0],[0]])
a_2 = np.array([[0],[1],[0]])
a_3 = np.array([[0],[0],[1]])
a_mat = np.concatenate((a_1, a_2, a_3), axis=1)
I = np.array([[0.04, 0.00, 0.00],
              [0.00, 0.05, 0.02],
              [0.00, 0.02, 0.05]])
I_inv = np.linalg.inv(I)
I_rw = 6.25E-05
I_wheels = I_rw*(a_1*np.transpose(a_1) + a_2*np.transpose(a_2) + a_3*np.transpose(a_3))

In [3]:
q = SX.sym("q", 4, 1)
w = SX.sym("w", 3, 1)
Omega = SX.sym("Omega", 3, 1)
V = SX.sym("V", 3, 1)
Q = SX.sym("Q", 4, 4)
Q[0,0] = 0
Q[0,1:] = -w
Q[1,1] = 0
Q[1:,0] = w
Q[1,2] = -w[2]
Q[1,3] = w[1]
Q[2,2] = 0
Q[2,1] = w[2]
Q[2,3] = w[0]
Q[3,3] = 0
Q[3,1] = -w[1]
Q[3,2] = w[0]
t = SX.sym('t', 1, 1) # Bit of a sketch workaround

In [5]:
q_dot = SX.sym("q_dot", 4, 1)
w_dot = SX.sym("w_dot", 3, 1)
Omega_dot = SX.sym("Omega_dot", 3, 1)
t_dot = SX.sym('t_dot', 1, 1) # Bit of a sketch workaround

In [6]:
'''
q_dot = (0.5)*(Q@q)
w_dot = I_inv@(-(Tmax/Vmax)*a_mat@V + (Tmax/Wmax)*a_mat@Omega
               - cross(w, (I@w) + (I_wheels@w) + I_rw*(a_mat@Omega)))
Omega_dot = (Tmax/(Vmax*I_rw))*V - (Tmax/(Wmax*I_rw))*Omega - a_mat.T@w
'''
f_expl = vertcat(
                (0.5)*(Q@q),
                -(Tmax/Vmax)*a_mat@V + (Tmax/Wmax)*a_mat@Omega - cross(w, (I@w) + (I_wheels@w) + I_rw*(a_mat@Omega)),
                (Tmax/(Vmax*I_rw))*V - (Tmax/(Wmax*I_rw))*Omega - a_mat.T@w,
                1*V[0]
                )
x = vertcat(q, w, Omega, t)
xdot = vertcat(q_dot, w_dot, Omega_dot, t_dot)
u = vertcat(V)
f_impl = xdot - f_expl

In [7]:
model = AcadosModel()
model.f_impl_expr = f_impl
model.f_expl_expr = f_expl
model.x = x
model.xdot = xdot
model.u = u
model.name = "reaction_wheel_spacecraft_model"

In [19]:
x0 = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
xf = np.array([0.7316889, 0.4544258, 0.2272129, 0.4544258, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

ocp = AcadosOcp()
ocp.model = model

N = 20
Tf = 300
nx = model.x.size()[0]
nu = model.u.size()[0]
ny = nx + nu
ny_e = nx

# set dimensions
ocp.dims.N = N

# set cost
ocp.cost.cost_type = 'EXTERNAL'
ocp.cost.cost_type_e = 'EXTERNAL'

ocp.model.cost_expr_ext_cost = 0
ocp.model.cost_expr_ext_cost_e = t

ocp.constraints.lbu = np.array([-6, -6, -6])
ocp.constraints.ubu = np.array([+6, +6, +6])
ocp.constraints.idxbu = np.array([0, 1, 2])
ocp.constraints.x0 = x0
ocp.constraints.lbx_e = xf
ocp.constraints.ubx_e = xf
ocp.constraints.idxbx_e = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0])
# set prediction horizon
ocp.solver_options.tf = Tf

# set options
ocp.solver_options.qp_solver = 'FULL_CONDENSING_QPOASES'#'PARTIAL_CONDENSING_HPIPM' # FULL_CONDENSING_QPOASES
ocp.solver_options.integrator_type = 'IRK'
ocp.solver_options.print_level = 0
ocp.solver_options.nlp_solver_type = 'SQP' # SQP_RTI, SQP
ocp.solver_options.globalization = 'MERIT_BACKTRACKING'
#ocp.solver_options.nlp_solver_max_iter = 5000
#ocp.solver_options.nlp_solver_tol_stat = 1e-6
#ocp.solver_options.levenberg_marquardt = 0.1
#ocp.solver_options.sim_method_num_steps = 15
#ocp.solver_options.qp_solver_iter_max = 100

In [25]:
ocp_solver = AcadosOcpSolver(ocp)

simX = np.ndarray((N+1, nx))
simU = np.ndarray((N, nu))

# for i, tau in enumerate(np.linspace(0, 1, N)):
#     ocp_solver.set(i, 'x', (1-tau)*x0 + tau*xf)
#     ocp_solver.set(i, 'u', np.array([0.1, 0.5]))

status = ocp_solver.solve()
ocp_solver.print_statistics() # encapsulates: stat = ocp_solver.get_stats("statistics")

if status != 0:
    raise Exception(f'acados returned status {status}.')




got cost_type_0: EXTERNAL, hessian_approx: 'GAUSS_NEWTON.'
GAUSS_NEWTON hessian is only supported for cost_types [NON]LINEAR_LS.
If you continue, acados will proceed computing the exact hessian for the cost term.
Note: There is also the option to use the external cost module with a numerical hessian approximation (see `ext_cost_num_hess`).
OR the option to provide a symbolic custom hessian approximation (see `cost_expr_ext_cost_custom_hess`).


got cost_type: EXTERNAL, hessian_approx: 'GAUSS_NEWTON.'
GAUSS_NEWTON hessian is only supported for cost_types [NON]LINEAR_LS.
If you continue, acados will proceed computing the exact hessian for the cost term.
Note: There is also the option to use the external cost module with a numerical hessian approximation (see `ext_cost_num_hess`).
OR the option to provide a symbolic custom hessian approximation (see `cost_expr_ext_cost_custom_hess`).


got cost_type_e: EXTERNAL, hessian_approx: 'GAUSS_NEWTON.'
GAUSS_NEWTON hessian is only supported for c

Exception: acados returned status 4.

In [12]:
ocp.constraints.lbx_e.shape[0]

11

In [13]:
ocp.constraints.ubx_e.shape[0]

11

In [23]:
u

SX([V_0, V_1, V_2])

In [24]:
xdot

SX([q_dot_0, q_dot_1, q_dot_2, q_dot_3, w_dot_0, w_dot_1, w_dot_2, Omega_dot_0, Omega_dot_1, Omega_dot_2, dt])

In [25]:
x

SX([q_0, q_1, q_2, q_3, w_0, w_1, w_2, Omega_0, Omega_1, Omega_2, t])

In [26]:
f_expl

SX(@1=0.5, @2=-0.000158833, @3=6.06699e-07, @4=0.02, @5=0.05, @6=6.25e-05, @7=((((@4*w_1)+(@5*w_2))+(@6*w_2))+(@6*Omega_2)), @8=((((@5*w_1)+(@4*w_2))+(@6*w_1))+(@6*Omega_1)), @9=(((0.04*w_0)+(@6*w_0))+(@6*Omega_0)), @10=2.54133, @11=0.00970718, [(-(@1*(((w_0*q_1)+(w_1*q_2))+(w_2*q_3)))), (@1*(((w_0*q_0)-(w_2*q_2))+(w_1*q_3))), (@1*(((w_1*q_0)+(w_2*q_1))+(w_0*q_3))), (@1*(((w_2*q_0)-(w_1*q_1))+(w_0*q_2))), (((@2*V_0)+(@3*Omega_0))-((w_1*@7)-(w_2*@8))), (((@2*V_1)+(@3*Omega_1))-((w_2*@9)-(w_0*@7))), (((@2*V_2)+(@3*Omega_2))-((w_0*@8)-(w_1*@9))), (((@10*V_0)-(@11*Omega_0))-w_0), (((@10*V_1)-(@11*Omega_1))-w_1), (((@10*V_2)-(@11*Omega_2))-w_2), V_0])

In [4]:
model_name = 'pendulum_ode'

# constants
M = 1. # mass of the cart [kg] -> now estimated
m = 0.1 # mass of the ball [kg]
g = 9.81 # gravity constant [m/s^2]
l = 0.8 # length of the rod [m]

# set up states & controls
x1      = SX.sym('x1')
theta   = SX.sym('theta')
v1      = SX.sym('v1')
dtheta  = SX.sym('dtheta')
t       = SX.sym('t')

x = vertcat(x1, theta, v1, dtheta, t)

F = SX.sym('F')
u = vertcat(F)

# xdot
x1_dot      = SX.sym('x1_dot')
theta_dot   = SX.sym('theta_dot')
v1_dot      = SX.sym('v1_dot')
dtheta_dot  = SX.sym('dtheta_dot')
dt          = SX.sym('dt')

xdot = vertcat(x1_dot, theta_dot, v1_dot, dtheta_dot, dt)

# dynamics
cos_theta = cos(theta)
sin_theta = sin(theta)
denominator = M + m - m*cos_theta*cos_theta
f_expl = vertcat(v1,
                 dtheta,
                 (-m*l*sin_theta*dtheta*dtheta + m*g*cos_theta*sin_theta+F)/denominator,
                 (-m*l*cos_theta*sin_theta*dtheta*dtheta + F*cos_theta+(M+m)*g*sin_theta)/(l*denominator),
                 1
                 )

f_impl = xdot - f_expl

model = AcadosModel()

model.f_impl_expr = f_impl
model.f_expl_expr = f_expl
model.x = x
model.xdot = xdot
model.u = u
model.name = model_name

In [24]:
x0 = np.array([0.0, 0, 0.0, 0.0, 0.0])
xf = np.array([0.7316889, 0.4544258, 0.2272129, 0.4544258])

ocp = AcadosOcp()
ocp.model = model

N = 20
Tf = 300
nx = model.x.size()[0]
nu = model.u.size()[0]
ny = nx + nu
ny_e = nx

# set dimensions
ocp.dims.N = N

# set cost
ocp.cost.cost_type = 'EXTERNAL'
ocp.cost.cost_type_e = 'EXTERNAL'

ocp.model.cost_expr_ext_cost = 0
ocp.model.cost_expr_ext_cost_e = t

ocp.constraints.lbu = np.array([-6])
ocp.constraints.ubu = np.array([+6])
ocp.constraints.idxbu = np.array([0])
ocp.constraints.x0 = x0
ocp.constraints.lbx_e = np.array([0, 0, 0, 0, 0])
ocp.constraints.ubx_e = np.array([0, 0, 0, 0, 0])
ocp.constraints.idxbx_e = np.array([0, 1, 2, 3, 4])
# set prediction horizon
ocp.solver_options.tf = Tf

# set options
ocp.solver_options.qp_solver = 'PARTIAL_CONDENSING_OSQP'#'PARTIAL_CONDENSING_HPIPM' # FULL_CONDENSING_QPOASES
ocp.solver_options.integrator_type = 'IRK'
ocp.solver_options.print_level = 0
ocp.solver_options.nlp_solver_type = 'SQP' # SQP_RTI, SQP
ocp.solver_options.globalization = 'MERIT_BACKTRACKING'
#ocp.solver_options.nlp_solver_max_iter = 5000
#ocp.solver_options.nlp_solver_tol_stat = 1e-6
#ocp.solver_options.levenberg_marquardt = 0.1
#ocp.solver_options.sim_method_num_steps = 15
#ocp.solver_options.qp_solver_iter_max = 100